In [3]:
#works but clean up
import requests
import urllib
import json
import csv
import time

#remember to hide and link to token later

base_url = "https://api.osf.io/v2/"
user_url = "users/7jwh6/nodes/"
project_url = "nodes/v3znw/"
returnChildren_url = "?embed=children"

#testProj = "osf.io/v3znw" 


token = (open("../apitoken.txt", "r")).read()


In [15]:
#function to make API call GET 
def send_get(URL, TOKEN):
    try:
        response = requests.get(
            url=URL,
            headers={
                "Authorization": "Bearer "+token,
                "Content-Type": "application/json",
            },
            allow_redirects=True
        )
        status = response.status_code
        #print("URL: {} , Status: {}".format(url, status))
        print("Link called by function: "+URL)

        if status < 300:
            return response.json()
            #res = response.json()
        else:
            print('Response HTTP Status Code: {status_code}'.format(
                status_code=response.status_code))
            print('Response HTTP Response Body: {content}'.format(
                content=response.content))
            
    except requests.exceptions.RequestException:
        print("HTTP Request Failed")

In [16]:
def send_get2(URL):
    #function that will paginate through results - hopefully
    try:
        response = requests.get(
            url=URL,
            headers={
                "Authorization": "Bearer "+token,
                "Content-Type": "application/json",
            },
            allow_redirects=True
        )
        status = response.status_code
        #print("URL: {} , Status: {}".format(url, status))
        print("Link called by function: "+URL)

        if status < 300:
            
            return response.json()
        
            #res = response.json()
            
        else:
            print('Response HTTP Status Code: {status_code}'.format(
                status_code=response.status_code))
            print('Response HTTP Response Body: {content}'.format(
                content=response.content))
            
    except requests.exceptions.RequestException:
        print("HTTP Request Failed")

In [18]:
cont_res = send_get2("https://api.osf.io/v2/nodes/9a5eh/contributors/?")
myJson = json.dumps(cont_res)
print(myJson)

Link called by function: https://api.osf.io/v2/nodes/9a5eh/contributors/?
{"links": {"first": null, "next": "https://api.osf.io/v2/nodes/9a5eh/contributors/?page=2", "last": "https://api.osf.io/v2/nodes/9a5eh/contributors/?page=3", "prev": null, "meta": {"per_page": 10, "total_bibliographic": 24, "total": 24}}, "data": [{"id": "9a5eh-3jzys", "attributes": {"unregistered_contributor": null, "permission": "admin", "index": 0, "bibliographic": true}, "relationships": {"users": {"links": {"related": {"meta": {}, "href": "https://api.osf.io/v2/users/3jzys/"}}}, "node": {"links": {"related": {"meta": {}, "href": "https://api.osf.io/v2/nodes/9a5eh/"}}}}, "type": "contributors", "embeds": {"users": {"data": {"type": "users", "attributes": {"active": true, "timezone": "America/New_York", "middle_names": "Erin", "given_name": "Christy", "date_registered": "2016-12-08T16:03:33.015000", "full_name": "Christy Erin Regenhardt", "suffix": "", "family_name": "Regenhardt", "locale": "en_US"}, "id": "3j

In [69]:
def checkForNext(jsonResult):
    #takes json from API get call and returns a new URL
    if jsonResult["links"]["next"] is not None:
        #print(jsonResult["links"]["next"])
        newURL = jsonResult["links"]["next"]
        print(newURL)
        return newURL
        
    else:
        global x 
        x = False
        print("no more links")
        

### Psudocode

Give send_get2() a URL with contributors
Return JSON 
Add each contributor from that URL to a list of contributors
Check if there is a "Next" url
If there is a "Next" url, pass it to send_get2()
Repeat 

In [74]:
# ~~~ Working Script. Much room for improvement. ~~~

#get contributor list from a project. Not easy to re-use because have to paginate manually.
#If reuse desired, must design for pagination. Also, edit so ends smoothly without just breaking on last.


#res = send_get("https://api.osf.io/v2/nodes/9taan/", token)
list_of_cont = []

cont_res = send_get("https://api.osf.io/v2/nodes/9a5eh/contributors/?page=3", token)
myJson = json.dumps(cont_res)
#print (myJson)

print(len(cont_res["data"]))


con_id = cont_res["data"][0]["embeds"]["users"]["data"]["id"]
print(json.dumps(con_id))



for i in range(0, (len(cont_res["data"]))): 
    con_id = cont_res["data"][i]["embeds"]["users"]["data"]["id"]
    permissions = cont_res["data"][i]["attributes"]["permission"]
    print("User ID: {}, Permission level: {}".format(con_id, permissions))
    contributor = [con_id, permissions]
    list_of_cont.append(contributor)

print(list_of_cont)

Link called by function: https://api.osf.io/v2/nodes/9a5eh/contributors/?page=3
4
"tymb5"
User ID: tymb5, Permission level: write
User ID: xgqtu, Permission level: write
User ID: nxfj8, Permission level: write
User ID: yj76v, Permission level: write
[['tymb5', 'write'], ['xgqtu', 'write'], ['nxfj8', 'write'], ['yj76v', 'write']]


In [77]:
#working! 
# Takes link to first page of contributors (assigned to my url)
# Goes through the json for each contributor "node" and returns their ID and permissions level

x = True

my_url = "https://api.osf.io/v2/nodes/9a5eh/contributors/?"

#firstResultJSON = send_get2(my_url)
list_of_contr = []

while x == True:
    jsonFromCall = send_get2(my_url)
        
    
    for i in range(0, (len(jsonFromCall["data"]))): 
        contributor_id = jsonFromCall["data"][i]["embeds"]["users"]["data"]["id"]
        permissions = jsonFromCall["data"][i]["attributes"]["permission"]
        print("User ID: {}, Permission level: {}".format(contributor_id, permissions))
        contributor = [contributor_id, permissions]
        list_of_contr.append(contributor)
    
    
    
    #linksFromJSON = jsonFromCall["data"][0]["embeds"]["users"]["data"]["links"]
    #print(linksFromJSON)
    #list_of_contr.append(linksFromJSON)
    
    #myJson = json.dumps(jsonFromInitialCall)
    #print (myJson)
    print(len(jsonFromInitialCall["data"]))
    
    print("~~~~~ THIS IS BREAK 1 ~~~~~")    
    
    my_url = checkForNext(jsonFromCall)

    #print(x)
    
    #con_id = jsonFromInitialCall["data"][0]["embeds"]["users"]["data"]["id"] #only the first ID from results
    #print(con_id)
    
    #print(json.dumps(con_id))
    time.sleep(2)

print("Done")    
print(list_of_contr)
# Almost works. Currently not changing to false in the function for some reason

Link called by function: https://api.osf.io/v2/nodes/9a5eh/contributors/?
User ID: 3jzys, Permission level: admin
User ID: 7jwh6, Permission level: admin
User ID: 6nywd, Permission level: admin
User ID: utb5k, Permission level: admin
User ID: 2kfwy, Permission level: write
User ID: f77p3, Permission level: write
User ID: hhdup, Permission level: write
User ID: vkthj, Permission level: write
User ID: 7595u, Permission level: admin
User ID: czszc, Permission level: write
10
~~~~~ THIS IS BREAK 1 ~~~~~
https://api.osf.io/v2/nodes/9a5eh/contributors/?page=2
Link called by function: https://api.osf.io/v2/nodes/9a5eh/contributors/?page=2
User ID: vutud, Permission level: write
User ID: bz3n5, Permission level: write
User ID: h6zc6, Permission level: write
User ID: 9j2ay, Permission level: write
User ID: 5kmjn, Permission level: write
User ID: m6wyb, Permission level: write
User ID: mec2f, Permission level: write
User ID: kfc3y, Permission level: write
User ID: 9t3xc, Permission level: write


In [86]:
#working! 
# Takes link to first page of contributors (assigned to my url)
# Goes through the json for each contributor "node" and returns their ID and permissions level


def getContributorIdAndPermissions(my_url):
    global x
    x = True

    #firstResultJSON = send_get2(my_url)
    list_of_contr = []

    while x == True:
        jsonFromCall = send_get2(my_url)


        for i in range(0, (len(jsonFromCall["data"]))): 
            contributor_id = jsonFromCall["data"][i]["embeds"]["users"]["data"]["id"]
            permissions = jsonFromCall["data"][i]["attributes"]["permission"]
            print("User ID: {}, Permission level: {}".format(contributor_id, permissions))
            contributor = [contributor_id, permissions]
            list_of_contr.append(contributor)



        #linksFromJSON = jsonFromCall["data"][0]["embeds"]["users"]["data"]["links"]
        #print(linksFromJSON)
        #list_of_contr.append(linksFromJSON)

        #myJson = json.dumps(jsonFromInitialCall)
        #print (myJson)
        print(len(jsonFromInitialCall["data"]))

        print("~~~~~ THIS IS BREAK 1 ~~~~~")    

        my_url = checkForNext(jsonFromCall)

        #print(x)

        #con_id = jsonFromInitialCall["data"][0]["embeds"]["users"]["data"]["id"] #only the first ID from results
        #print(con_id)

        #print(json.dumps(con_id))
        time.sleep(2)

    print("Done")    
    #print(list_of_contr)
    return list_of_contr
    # Almost works. Currently not changing to false in the function for some reason

In [87]:
my_url = "https://api.osf.io/v2/nodes/9a5eh/contributors/?"
print(getContributorIdAndPermissions(my_url))

Link called by function: https://api.osf.io/v2/nodes/9a5eh/contributors/?
User ID: 3jzys, Permission level: admin
User ID: 7jwh6, Permission level: admin
User ID: 6nywd, Permission level: admin
User ID: utb5k, Permission level: admin
User ID: 2kfwy, Permission level: write
User ID: f77p3, Permission level: write
User ID: hhdup, Permission level: write
User ID: vkthj, Permission level: write
User ID: 7595u, Permission level: admin
User ID: czszc, Permission level: write
10
~~~~~ THIS IS BREAK 1 ~~~~~
https://api.osf.io/v2/nodes/9a5eh/contributors/?page=2
Link called by function: https://api.osf.io/v2/nodes/9a5eh/contributors/?page=2
User ID: vutud, Permission level: write
User ID: bz3n5, Permission level: write
User ID: h6zc6, Permission level: write
User ID: 9j2ay, Permission level: write
User ID: 5kmjn, Permission level: write
User ID: m6wyb, Permission level: write
User ID: mec2f, Permission level: write
User ID: kfc3y, Permission level: write
User ID: 9t3xc, Permission level: write


In [ ]:

#testJson = json.dumps(cont_ress)
#print(len(cont_ress["links"]))
#print(cont_ress["links"]["last"])
#print(testJson)

print("~~~~~ THIS IS BREAK 1 ~~~~~")
cont_ress = send_get2("https://api.osf.io/v2/nodes/9a5eh/contributors/?")

while True:
    if cont_ress["links"]["next"] is not None:
        send_get2(cont_ress["links"]["next"])
        print(cont_ress["links"]["next"])

    else:
        print("This is null")
        break

In [13]:
print(len(list_of_cont))
print(list_of_cont)

23
[['3jzys', 'admin'], ['7jwh6', 'admin'], ['6nywd', 'admin'], ['utb5k', 'admin'], ['2kfwy', 'write'], ['f77p3', 'write'], ['hhdup', 'write'], ['vkthj', 'write'], ['7595u', 'admin'], ['czszc', 'write'], ['vutud', 'write'], ['bz3n5', 'write'], ['h6zc6', 'write'], ['9j2ay', 'write'], ['5kmjn', 'write'], ['m6wyb', 'write'], ['mec2f', 'write'], ['kfc3y', 'write'], ['9t3xc', 'write'], ['nnjvk', 'write'], ['tymb5', 'write'], ['xgqtu', 'write'], ['nxfj8', 'write']]


In [76]:
print(list_of_contr)

[['3jzys', 'admin'], ['7jwh6', 'admin'], ['6nywd', 'admin'], ['utb5k', 'admin'], ['2kfwy', 'write'], ['f77p3', 'write'], ['hhdup', 'write'], ['vkthj', 'write'], ['7595u', 'admin'], ['czszc', 'write'], {'profile_image': 'https://secure.gravatar.com/avatar/2cca46b5153caa027d797f02250a645c?d=identicon', 'html': 'https://osf.io/tymb5/'}, ['vutud', 'write'], ['bz3n5', 'write'], ['h6zc6', 'write'], ['9j2ay', 'write'], ['5kmjn', 'write'], ['m6wyb', 'write'], ['mec2f', 'write'], ['kfc3y', 'write'], ['9t3xc', 'write'], ['nnjvk', 'write'], {'profile_image': 'https://secure.gravatar.com/avatar/2cca46b5153caa027d797f02250a645c?d=identicon', 'html': 'https://osf.io/tymb5/'}, ['tymb5', 'write'], ['xgqtu', 'write'], ['nxfj8', 'write'], ['yj76v', 'write'], {'profile_image': 'https://secure.gravatar.com/avatar/2cca46b5153caa027d797f02250a645c?d=identicon', 'html': 'https://osf.io/tymb5/'}]


In [25]:
list_of_cont = []

url = "https://api.osf.io/v2/nodes/9taan/contributors/"

try:
    #while loop makes paging through results possible
    while True:
        response = requests.get(
            url=url,
            headers={
                "Authorization": "Bearer "+token,
                "Content-Type": "application/json",
            },
            #data=json.dumps(request_json),
            allow_redirects=True
        )
        status = response.status_code
        print("URL: {}, Status: {}".format(url, status))

        if status < 300:
            res = response.json()
            print ("Length of list:")
            print (len(res["links"]))
            print (len(res["data"]))
            #print(json.dumps(res))
            for i in range(0, (len(res["data"]))): 
                con_id = res["data"][i]["embeds"]["users"]["data"]["id"]
                permissions = res["data"][i]["attributes"]["permission"]
                print("User ID: {}, Permission level: {}".format(con_id, permissions))
               
                
            #make list of lists
                contributor = [con_id, permissions]
                list_of_cont.append(contributor)
                
                


    else:
        print("Response HTTP Status Code: {status_code}".format(status_code=response.status_code))
        print("Response HTTP Response Body: {content}".format(content=response.content))
except requests.exceptions.RequestException:
    print("HTTP Request Failed")
    
#text_file.close()

print(list_of_cont)

print("End of cell")

URL: https://api.osf.io/v2/nodes/9taan/contributors/, Status: 200
Length of list:
5
10
User ID: 3jzys, Permission level: admin
User ID: 7jwh6, Permission level: admin
User ID: 6nywd, Permission level: admin
User ID: utb5k, Permission level: admin
User ID: mr99g, Permission level: write
User ID: 2kfwy, Permission level: write
User ID: f77p3, Permission level: write
User ID: hhdup, Permission level: write
User ID: vkthj, Permission level: write
User ID: 7595u, Permission level: admin
URL: https://api.osf.io/v2/nodes/9taan/contributors/, Status: 200
Length of list:
5
10
User ID: 3jzys, Permission level: admin
User ID: 7jwh6, Permission level: admin
User ID: 6nywd, Permission level: admin
User ID: utb5k, Permission level: admin
User ID: mr99g, Permission level: write
User ID: 2kfwy, Permission level: write
User ID: f77p3, Permission level: write
User ID: hhdup, Permission level: write
User ID: vkthj, Permission level: write
User ID: 7595u, Permission level: admin
URL: https://api.osf.io/v2

KeyboardInterrupt: 

In [14]:
#Gets metadata for a given project and top 2 levels of components
url1 = base_url + project_url + returnChildren_url

sub_links = []

#api call on project
res = send_get(url1, token)

title = res["data"]["attributes"]["title"]
get_children = res["data"]["embeds"]["children"]["data"]

print("Title of Project: "+title)

#get info on each top-level component
for i in range(0, 2):
    #sub_com = res["data"]["embeds"]["children"]["data"][i]["attributes"]["title"]
    sub_com = get_children[i]["attributes"]["title"]
    sub_des = get_children[i]["attributes"]["description"]
    sub_link = get_children[i]["links"]["self"]
    
    print("Call #"+str(i))
    print("Title of Component: {}, Description of Component: {}, Link: {}".format(sub_com, sub_des, sub_link))
    
    sub_links.append(sub_link) #left in case I want an example of pulling things out

    #api call on each top-level component to get info on second-level components
    sub_res = send_get(sub_link+returnChildren_url, token) 
    sub_com_child_data = sub_res["data"]["embeds"]["children"]["data"]
    print(len(sub_com_child_data))
    
    #get info on each second-level component
    for u in range(0, len(sub_com_child_data)):
        title_test1 = sub_com_child_data[u]["attributes"]["title"]
        desc_test1 = sub_com_child_data[u]["attributes"]["description"]
        link_test1 = sub_com_child_data[u]["links"]["self"]
        print("Title of Component lvl 2: {}, Description of Component lvl 2: {}, Link to Component lvl 2: {}".format(title_test1, desc_test1, link_test1))
#    print(sub_com_child_data)
    
    

#    print(json.dumps(res))


print(sub_links)


    
    
print("End of cell")

Link called by function: https://api.osf.io/v2/nodes/v3znw/?embed=children
Title of Project: Sample Top Level Project
Call #0
Title of Component: SubComponent, Description of Component: Description box on subComponent level, Link: https://api.osf.io/v2/nodes/ufrbm/
Link called by function: https://api.osf.io/v2/nodes/ufrbm/?embed=children
2
Title of Component lvl 2: Sub A, Description of Component lvl 2: 12345a, Link to Component lvl 2: https://api.osf.io/v2/nodes/94pm3/
Title of Component lvl 2: Sub_B, Description of Component lvl 2: 12345b, Link to Component lvl 2: https://api.osf.io/v2/nodes/zhfd2/
Call #1
Title of Component: Another_SubComponent, Description of Component: This is a component description box for the second sub component, Link: https://api.osf.io/v2/nodes/ktbzk/
Link called by function: https://api.osf.io/v2/nodes/ktbzk/?embed=children
0
['https://api.osf.io/v2/nodes/ufrbm/', 'https://api.osf.io/v2/nodes/ktbzk/']
End of cell


In [12]:
#Gets metadata for all a user's projects/components and prints Title, description, and link to a CSV
#works but needs some cleaning to make better and more reusable
#Incorporate send_get() function

url = base_url + user_url

linesOfText = []

try:
    #while loop makes paging through results possible
    while True:
        response = requests.get(
            url=url,
            headers={
                "Authorization": "Bearer "+token,
                "Content-Type": "application/json",
            },
            #data=json.dumps(request_json),
            allow_redirects=True
        )
        status = response.status_code
        print("URL: {}, Status: {}".format(url, status))

        if status < 300:
            res = response.json()
            print ("Length of list:")
            print (len(res["links"]))
            print (len(res["data"]))
            #print(json.dumps(res))
            for i in range(0, (len(res["data"]))):
                
                title = res["data"][i]["attributes"]["title"]
                description = res["data"][i]["attributes"]["description"]
                componentLink = res["data"][i]["links"]["html"]
                print("Title: {}, Description: {}".format(title, description))
                print("Link to component: {}".format(componentLink))
                
            #make list of lists to print to CSV that I will sift through at next step
                variableLines = [title, description, componentLink]
                linesOfText.append(variableLines)
                
                print("Item: "+ str(i))
                print(res["links"]["next"])
                if i == (len(res["data"])-1):
                    url = res["links"]["next"]
                    print(url)
                #print(res["data"][i]["links"])


    else:
        print("Response HTTP Status Code: {status_code}".format(status_code=response.status_code))
        print("Response HTTP Response Body: {content}".format(content=response.content))
except requests.exceptions.RequestException:
    print("HTTP Request Failed")
    
#text_file.close()
print("End of cell")


print(linesOfText)

URL: https://api.osf.io/v2/nodes/v3znw/?embed=children, Status: 200
Length of list:


KeyError: 'links'

In [3]:
#Write the list of lists from API call to a csv
with open("test.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(linesOfText)

In [ ]:
#Next Step involves grabbing the files based on the names listed in the description field of the CSV

In [ ]:
#define project URL to stay within
#somehow need to include all components of this project
projectURL = "https://api.osf.io/v2/nodes/v3znw/?format=json"

#Connect to API, logged in with credentials, go into specific project


#

Each page response for a user search has a header section "links" and a "data" section.
Within data is a list of 10 dictionaries max (because 10 per page)
These have "links" -> "html"
"attributes" -> "title"

title = ["data"][i]["attributes"]["title"]
description = ["data"][i]["attributes"]["description"]

Each page response for a project has "data" at the top level
"data" is the key to a value of dictionaries: "relationships", "links", "attributes" (this is probably the one you want), "type" and "id"

In [74]:
r = requests.get(url)

print(r.url)
print(r)

resultdata = r.json()
resultdata



https://api.osf.io/v2/users/7jwh6/nodes/?page=5
<Response [404]>


{'errors': [{'detail': 'Invalid page.'}]}

In [ ]:
url2 = "https://api.osf.io/v2/users/7jwh6/"
p = requests.get(
{
        "data": {
            "id": "7jwh6"
            "type": "users"
            
        }
        
    }
)

Some API usage guidelines and examples are here. About half-way down there are some PUT/POST/PATCH examples. 

For the authorization, it goes in the header section of the request. So something like:

headers = {
     "Authorization": my_auth
}

With my_auth being your token.

---

An example was just posted here about the authorization piece:

https://osf.io/y9jdt/wiki/Examples/

In [12]:
t = 7 
print(t)

#person = input("what's your name? ")
#print("Hi {}".format(person))



7
